In [1]:
import collections
import gensim
import heapq
import json
import matplotlib.pyplot as plt
import numpy as np
import os
import time

from matplotlib import offsetbox
from scipy.sparse import csr_matrix
from sklearn import cluster, datasets, decomposition, ensemble, lda, manifold, random_projection, grid_search
from sklearn.decomposition import TruncatedSVD
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC, SVC
from scipy import sparse as sp

%matplotlib inline

In [6]:
def write_evaluation(tuples, filename):
    f = open(filename, 'w+')
    f.write('Id,Prediction\n')
    for t in tuples:
        f.write(str(t[0]) + ',' + str(t[1]) + '\n')

In [13]:
def load_sparse_data(filename, num_lines):
    """
    Function to load sparse data.
    """
    inverted_index = collections.defaultdict(set)
    
    sparse_indptr = [0]
    sparse_indices = []
    sparse_data = []
    vocabulary = {}

    print 'Reading data.'
    for line_num, line in enumerate(open(filename)):
        new_row = [(idx,float(prob)) for idx, prob in enumerate(line.strip().split(',')) if float(prob) > 0.0]
        for i,p in new_row:
            sparse_indices.append(i)
            sparse_data.append(p)
            inverted_index[i].add(line_num)
        sparse_indptr.append(len(sparse_indices))

        if line_num % 450 == 0:
            print 100.0 * line_num / num_lines, '%'
    print 100.0 * line_num / num_lines, '%'
    print 'Done reading data.'

    sparse_matrix = csr_matrix((sparse_data, sparse_indices, sparse_indptr), dtype=float)
    return sparse_matrix, inverted_index

In [8]:
def sparse_where(sparse_matrix, num):
    """
    np.where() for a sparse matrix. Returns a set of indices.
    """
    return set(np.where(sparse_matrix[num,:].toarray())[1].tolist())

In [9]:
def load_clustering(fname):
    cluster_dict = []
    cluster_inv_idx = collections.defaultdict(list)
    
    todo
    
    return cluster_dict, cluster_inv_idx

# Load Data

In [14]:
NUM_SPEECHES = 2740
speech_graph, inverted_graph = load_sparse_data('speech_graph.csv', NUM_SPEECHES)
print speech_graph.shape

Reading data.
0.0 %
16.4233576642 %
32.8467153285 %
49.2700729927 %
65.6934306569 %
82.1167883212 %
98.5401459854 %
99.9635036496 %
Done reading data.
(2740, 2740)


In [ ]:
predicted_label, predicted_label_inv_idx = load_clustering('hacky1.csv')

# Hints

### Actual cluster sizes

In [2]:
header = False
actual_cluster_sizes = []
for line in open('challenge-2-label-hint.csv'):
    if header:
        header = False
        continue
    actual_cluster_sizes.append(int(line.strip().split(',')[1]))

actual_cluster_sizes.sort()
print actual_cluster_sizes
print len(actual_cluster_sizes)

[3, 11, 12, 14, 14, 17, 19, 20, 21, 26, 27, 28, 30, 35, 41, 42, 42, 42, 43, 44, 45, 53, 54, 58, 58, 60, 64, 84, 87, 89, 114, 117, 130, 134, 159, 282, 294, 327]
38


### Actual cluster seeds

In [5]:
header = False
actual_debates = {}
for line in open('actual-debate-labels.csv'):
    if header:
        header = False
        continue
    parts = line.strip().split(',')
    actual_debates[int(parts[0])] = int(parts[1])
    
print len(actual_debates)

38


# Evaluate the cluster sizes we found

# Fix the three largest clusters (9 incorrect)

In [ ]:
speech_graph_copy_trimmed = speech_graph[sorted(remaining),:]
print speech_graph_copy_trimmed.shape
speech_graph_copy_trimmed = speech_graph_copy_trimmed[:,sorted(remaining)]
print speech_graph_copy_trimmed.shape

In [ ]:
spectral_labels = cluster.spectral_clustering(speech_graph_copy_trimmed, n_clusters=2)
plt.hist(spectral_labels, bins=10)
print [list(spectral_labels).count(i) for i in range(2)]

In [ ]:
write_evaluation([(i, d[i]) for i in range(NUM_SPEECHES)], 'fix.csv')